In [58]:
%pip install pandas -U -q
%pip install numpy -U -q
%pip install yfinance -U -q

In [59]:
from typing import List, Dict

import pandas as pd
import numpy as np
import yfinance as yf

## Unir as ações que compõem os índices IBOV e IDIV

In [60]:
def get_tickers_from_market_index(source: str):
  market_index = pd.read_csv(source)
  market_index.drop(columns="Part. (%)", inplace=True)
  market_index.dropna(inplace=True)
  return market_index.iloc[:, 0].to_list()

def merge_tickers_from_market_indexes(*args: List[str]) -> List[str]:
  result = set()

  for market_index in args:
    result = result.union(market_index)

  return list(result)

In [61]:
ibov_composition = get_tickers_from_market_index("./assets/IBOV-composition.csv")
ibov_composition

In [62]:
idiv_composition = get_tickers_from_market_index("./assets/IDIV-composition.csv")
idiv_composition

## Base de dados com com todos os atributos que compõem o fluxo de caixa

In [63]:
tickers = merge_tickers_from_market_indexes(ibov_composition, idiv_composition)

In [64]:
def format_tickers_to_yfinance(tickers: List[str]) -> List[str]:
  return [f"{ticker}.SA" for ticker in tickers]

tickers = format_tickers_to_yfinance(tickers)

In [65]:
cash_flow_data = pd.DataFrame()

for ticker in tickers:
    cashflow = yf.Ticker(ticker).quarterly_cashflow
    
    # Define o ticker como a coluna principal no MultiIndex
    cashflow.columns = pd.MultiIndex.from_product([[ticker], cashflow.columns])
    
    if cash_flow_data.empty:
        cash_flow_data = cashflow
        continue

    cash_flow_data = pd.concat([cash_flow_data, cashflow], axis=1)


In [75]:
cash_flow_data.dropna(inplace=True)
cash_flow_data

In [67]:
cash_flow_data.to_csv("./assets/brazilian_companies_complete_cashflow.csv")

## Base de dados  com os atributos mais relevantes para o cálculo do fluxo de caixa livre

O critério de escolha dos atributos mais relevantes é feito da seguinte maneira:

1. Cálculo da correlação de todos os atributos em relação ao atributo "Free Cash Flow" para cada ativo da base de dados;
2. Obter todos os atributos classificados como correlação forte ($cor \geq 0.7$) de cada ativo;
3. Obter os 3 atributos com maior frequência com base na etapa anterior.

Considerações importantes:

- Não foi escolhido o critério da correlação média dos atributos em relação ao "Free Cash Flow", pois pelo fato da média não ser uma medida resistente (sensível a outliers). Dessa forma, isso pode afetar o resultado de algoritmos de previsão do "Free Cash Flow", uma vez que pode ocorrer de um dos atributos selecionados como "mais relevantes" não explique a variação do "Free Cash Flow" de forma apropriada.

In [68]:
tickers = list(set([ticker for ticker, _ in cash_flow_data.columns]))
tickers

In [69]:
correlation_results = []

for ticker in tickers:
  df = cash_flow_data[ticker].transpose()
  correlation_matrix = df.corr()["Free Cash Flow"]

  # Filtra os atributos com a correlação média >= 0.7 em relação ao atributo "Free Cash Flow"
  high_corr = correlation_matrix[correlation_matrix >= 0.7]

  # Atributos com correlação forte com a coluna "Free Cash Flow"
  most_relevant = high_corr.sort_values().index.to_list()
  correlation_results.append(most_relevant)


In [70]:
import itertools
import collections


def frequency_counter(matrix: List[List[any]], n: int | None = None) -> Dict[any, int]:
    # flatten the list using chain()
    flat_list = list(itertools.chain.from_iterable(matrix))

    # count the frequencies using Counter()
    frequencies = collections.Counter(flat_list)

    if not n:
        n = len(frequencies)
    
    # Get the N most frequent attributes
    frequencies = {
        attr: frequency
        for rank, (attr, frequency) in enumerate(sorted(frequencies.items(), key=lambda item: item[1], reverse=True))
        if rank < n
    }

    return frequencies

In [71]:
frequency_counter(correlation_results)

In [76]:
most_relevant = frequency_counter(correlation_results, 4)
most_relevant

In [77]:
filter = cash_flow_data.index.isin(most_relevant)
cash_flow_data = cash_flow_data[filter]
cash_flow_data

In [78]:
cash_flow_data.to_csv("./assets/brazilian_companies_most_relevant_attr_for_free_cash_flow.csv")